In [1]:
import wallycore as wally

In [2]:
def buildTransaction(tx_inputs, tx_outputs):
    tx = wally.tx_init(2, 0, 1, 2) # version 2, locktime 0, 1 input, 2 outputs

    for tx_input in tx_inputs:
        wally.tx_add_input(tx, tx_input)

    for tx_output in tx_outputs:
        wally.tx_add_output(tx, tx_output)
        
    return tx
    
def buildInput(tx_input_hex, tx_input_vout, sequence):
    tx_input_byte = wally.hex_to_bytes(tx_input_hex)[::-1]
    return wally.tx_elements_input_init(tx_input_byte, tx_input_vout, sequence, None, None, None, None, None, None, None, None, None)

def buildOutput(value, asset_id, script_byte):
    confidential_tx_value_byte = wally.tx_confidential_value_from_satoshi(value)
    return wally.tx_elements_output_init(script_byte, asset_id, confidential_tx_value_byte, None, None, None)

In [3]:
ELEMENTS_UNBLINDED_TX_PREFIX = 0x01    

tx_input_hex = "53174bc15eef5b16522b5fe45cf2e147e829990dd72454ab7d2a493e220e4c36"
tx_input_vout = 0
asset_id = bytearray([ELEMENTS_UNBLINDED_TX_PREFIX]) + wally.hex_to_bytes("9e56d67dee0fbe395374572d2a389e56659380e2b58ad43081e0d3ddcab1a0f7")[::-1]
script = wally.hex_to_bytes("76a9144f844a62154987b501dccfd6c504596759e9f24888ac")
sequence = 0xffffffff

tx_inputs = [buildInput(tx_input_hex, tx_input_vout, sequence)]
tx_outputs = [buildOutput(12345, asset_id, script), buildOutput(500, asset_id, None)]

tx = buildTransaction(tx_inputs, tx_outputs)

tx_hex = wally.tx_to_hex(tx, 0)
print(tx_hex)

020000000001364c0e223e492a7dab5424d70d9929e847e1f25ce45f2b52165bef5ec14b17530000000000ffffffff0201f7a0b1caddd3e08130d48ab5e2809365569e382a2d57745339be0fee7dd6569e010000000000003039001976a9144f844a62154987b501dccfd6c504596759e9f24888ac01f7a0b1caddd3e08130d48ab5e2809365569e382a2d57745339be0fee7dd6569e0100000000000001f4000000000000


In [4]:
def signInputs(tx, private_keys, prev_out_scripts, sighashes):  
    for index, (private_key, prev_out_script, sighash) in enumerate(zip(private_keys, prev_out_scripts, sighashes)):
        hash_to_sign = wally.tx_get_elements_signature_hash(tx, index, prev_out_script, None, sighash, 0)
        public_key  = wally.ec_public_key_from_private_key(private_key)
        signature = wally.ec_sig_from_bytes(private_key, hash_to_sign, wally.EC_FLAG_ECDSA | wally.EC_FLAG_GRIND_R)
        script_sig = wally.scriptsig_p2pkh_from_sig(public_key, signature, sighash) 
        wally.tx_set_input_script(tx, index, script_sig)

In [5]:
private_key = wally.hex_to_bytes("33a61b1527bffa1a86f851a1c8d346e30f716646c128c074bfe258bb02021c79")
signInputs(tx, [private_key], [script], [wally.WALLY_SIGHASH_ALL])

tx_hex = wally.tx_to_hex(tx, 0)
print(tx_hex)

020000000001364c0e223e492a7dab5424d70d9929e847e1f25ce45f2b52165bef5ec14b1753000000006a47304402207250c51715d26685a0318506761189ca57c4f1eb419237d81bd17cb1b7bbac380220390400d16d4788fc9a87f19a943b6e1647b0ea5b54c212b36b0ef4f06e3b7b090121038a25c2d1dad078c421cd991688ff9d4b32a11697e377e2d065d358062fa2376dffffffff0201f7a0b1caddd3e08130d48ab5e2809365569e382a2d57745339be0fee7dd6569e010000000000003039001976a9144f844a62154987b501dccfd6c504596759e9f24888ac01f7a0b1caddd3e08130d48ab5e2809365569e382a2d57745339be0fee7dd6569e0100000000000001f4000000000000
